In [1]:
%run 00.style.ipynb

/Users/piazza/Playground/tweet-in-love


In [2]:
import spacy
from spacy.tokens import DocBin

from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import TransformerMixin
from typing import List, Tuple, Iterable

from tweet_in_love.settings import GlobalSettings, ModelSettings
from tweet_in_love.preprocessor import PreProcessor

settings = GlobalSettings()

In [3]:
settings.train_csv_path

PosixPath('data/train/train.csv')

# Load data

In [4]:
df_train = pd.read_csv(settings.train_csv_path)
df_test = pd.read_csv(settings.test_csv_path)

In [5]:
df_train.sample(5)

,content,sentiment
22038,My computer died,sadness
8868,"I am sooooo happy! Finally, Kean Cipriano repl...",love
1092,"gooooooodmorning world(: god bless, and have a...",worry
6631,@emmarler i am jealous of your mom talking to ...,worry
25462,@lewisking don't you procrastinate! (like I ...,worry


In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
df_train.head()

,content,sentiment
0,NEW YORK DOLLS TOMORROW NIGHT @ THE DEPOT CANC...,fun
1,@cosRobPerkins You're the third person to ask ...,surprise
2,"@thomaskattus you asked about my SF schedule, ...",neutral
3,SWAY SWAY BABY zommgg love it need more screa...,happiness
4,@LaDyBuG21 Thanks,neutral


# Preprocess

Just load the data in spacy's format

In [9]:
preprocessor = PreProcessor(
    nlp=nlp,
    classes=df_train.sentiment,
)

In [10]:
train_docs = preprocessor.df_to_doc(df=df_train)

In [11]:
test_docs = preprocessor.df_to_doc(df=df_test)

In [12]:
preprocessor.save_docs(train_docs, test_docs, tag="web_sm")

(PosixPath('data/train/train_web_sm.spacy'),
 PosixPath('data/test/test_web_sm.spacy'))

In [13]:
train_docs[0].cats

{'fun': 1,
 'boredom': 0,
 'relief': 0,
 'anger': 0,
 'surprise': 0,
 'happiness': 0,
 'neutral': 0,
 'love': 0,
 'sadness': 0,
 'hate': 0,
 'enthusiasm': 0,
 'worry': 0}

----
# Take a look at the preprocessed data

In [14]:
sample = preprocessor.df_to_doc(df=df_train.sample(10))

In [15]:
for doc in sample:
    print(doc)
    print(doc.ents)
    print([(token.lemma_) for token in doc])

    print([(token.vector_norm) for token in doc])
    print()

my girlies @VeeVeeBOMBSHELL &amp; @ReeseCromwell r leavin me today  Vanessa, I'll c u when u'r black *I mean back and Reese, I'll c u Sunday
(girlies @VeeVeeBOMBSHELL &, @ReeseCromwell, today, Reese, Sunday)
['my', 'girlie', '@VeeVeeBOMBSHELL', '&', 'amp', ';', '@ReeseCromwell', 'r', 'leavin', 'I', 'today', ' ', 'Vanessa', ',', 'I', "'ll", 'c', 'u', 'when', "u'r", 'black', '*', 'I', 'mean', 'back', 'and', 'Reese', ',', 'I', "'ll", 'c', 'u', 'Sunday']
[9.148404, 6.9401517, 8.537192, 9.016395, 7.8125644, 9.812474, 7.7398686, 6.605575, 6.785845, 7.818848, 9.122936, 8.097483, 7.952559, 7.6849937, 7.36979, 7.771745, 7.294192, 6.14944, 9.8892975, 6.5152082, 6.4241886, 7.3274245, 8.844927, 7.944716, 6.912108, 9.016117, 7.1334057, 7.213392, 6.9743543, 7.7682514, 7.029519, 7.044068, 9.256737]

Good Morning Sunshines!
(Good Morning Sunshines,)
['Good', 'Morning', 'Sunshines', '!']
[7.8765597, 7.4672213, 6.9710283, 10.281048]

urgh, over slept for work, still done no revision and im SO snappy tod

- The vanilla pretrained model is already able to detect `@mentions`.
  It also calculates their vector.
- `#hashtags` are treated as words, removing the `#`

# Add a sentiment score

In [50]:
nlp2 = spacy.load("en_core_web_sm")

# Add spaCyTextBlob-based sentiment analysis
nlp2.add_pipe("spacytextblob")

In [51]:
preprocessor = PreProcessor(
    nlp=nlp2,
    classes=df_train.sentiment,
)

In [52]:
sample = preprocessor.df_to_doc(df=df_train.sample(10))

In [54]:
for doc in sample:
    print(doc)
    print(doc.ents)
    print(
        f"pola:{doc._.polarity} subj:{doc._.subjectivity}\nassess:{doc._.assessments}"
    )
    print("..")
    print([(token.lemma_) for token in doc])
    print([(token.vector_norm) for token in doc])

    print("--\n")

bought beer today
(today,)
pola:0.0 subj:0.0
assess:[]
..
['buy', 'beer', 'today']
[8.238936, 7.4964523, 10.483919]
--

@honytawk why do we need to wait another year for toy story 3?!
(another year, 3)
pola:0.0 subj:0.0
assess:[]
..
['@honytawk', 'why', 'do', 'we', 'need', 'to', 'wait', 'another', 'year', 'for', 'toy', 'story', '3', '?', '!']
[6.6258135, 8.821145, 9.228143, 9.22015, 7.450351, 8.512473, 7.317643, 8.592593, 6.8815374, 6.471072, 6.466468, 6.1588793, 8.160021, 9.446849, 10.280108]
--

@torriRAWR OMG apparantley green day are touring here in december..we are so going
(green day, december)
pola:-0.2 subj:0.3
assess:[(['green'], -0.2, 0.3, None)]
..
['@torriRAWR', 'OMG', 'apparantley', 'green', 'day', 'be', 'tour', 'here', 'in', 'december', '..', 'we', 'be', 'so', 'go']
[6.716484, 8.642971, 6.7387495, 7.5328865, 7.077684, 8.45785, 7.413875, 8.412568, 8.031971, 9.421901, 7.8196898, 7.830092, 7.292568, 8.738072, 7.3948]
--

&quot;To boldly go where no one has gone before...&quo

- Vanilla TextBlob is already providing some nice data points to build a baseline classifier

In [21]:
train_docs = preprocessor.df_to_doc(df=df_train)

In [22]:
test_docs = preprocessor.df_to_doc(df=df_test)

In [23]:
preprocessor.save_docs(train_docs, test_docs, tag="txtblob_sm")

(PosixPath('data/train/train_txtblob_sm.spacy'),
 PosixPath('data/test/test_txtblob_sm.spacy'))

# Simplify the text
I'm gonna try to simplify the text by:
- translate emojis
- spell check the text
- remove mentions
- remove stopwords
- remove websites
- extract lemma
- set lowercase

In [6]:
nlpc = spacy.load("en_core_web_sm")

## Translate emojis

In [7]:
from spacymoji import Emoji

emoji = Emoji(nlpc)
nlpc.add_pipe("emoji", first=True)

In [8]:
doc = nlpc("This is a test 😻 👍🏿")
assert doc._.has_emoji == True
assert doc[2:5]._.has_emoji == True
assert doc[0]._.is_emoji == False
assert doc[4]._.is_emoji == True
assert doc[5]._.emoji_desc == "thumbs up dark skin tone"
assert len(doc._.emoji) == 2
assert doc._.emoji[1] == ("👍🏿", 5, "thumbs up dark skin tone")

## Spell-check the texts

Add a [contextual spell checker](https://spacy.io/universe/project/contextualSpellCheck) to improve the classification


In [9]:
import contextualSpellCheck

nlpc.add_pipe("contextual spellchecker")

In [10]:
preprocessor = PreProcessor(
    nlp=nlpc,
    classes=df_train.sentiment,
)

checked_docs = preprocessor.df_to_doc(df=df_train.head())

In [11]:
# compare original and spell-checked tweets
for i in checked_docs:
    print(i)
    print(i._.outcome_spellCheck)
    print()

NEW YORK DOLLS TOMORROW NIGHT @ THE DEPOT CANCELLED   REFUNDS AVAILABLE AT POINT OF PURCHASE.
NEW YORK DOLLS TOMORROW NIGHT @ THE DEPOT CANCELLED   AND AVAILABLE AT POINT OF PURCHASE.

@cosRobPerkins You're the third person to ask me out on Mon afternoon ;) I'm so sorry, but I'm helping tutor my friend for C1/C2  xx
@cosRobPerkins You're the third person to ask me out on Mon afternoon ;) I'm so sorry, but I'm helping tutor my friend for C1/C2 .

@thomaskattus you asked about my SF schedule, dahling...maybe next time
@thomaskattus you asked about my SF schedule, and...maybe next time

SWAY SWAY BABY zommgg love it  need more screamo tho boys
SWAY SWAY BABY You love it  need more from the boys

@LaDyBuG21 Thanks




## Detect mentions

In [12]:
try:
    Token.remove_extension("is_mention")
except:
    pass

In [13]:
from spacy.tokens import Token, Doc

# add extension to identify mentions

# Define getter function
def is_mention(token):
    return token.text.startswith("@")


# Set extension on the Span with getter
Token.set_extension("is_mention", getter=is_mention)

In [14]:
for t in nlpc("yo to @you"):
    print(t._.is_mention)

False
False
True


## Remove websites

In [15]:
try:
    Token.remove_extension("is_web")
except:
    pass

In [16]:
from spacy.tokens import Token, Doc

# add extension to identify mentions

# Define getter function
def is_web(token):
    return token.text.startswith("http")


# Set extension on the Span with getter
Token.set_extension("is_web", getter=is_web)

In [17]:
for t in nlpc("best http://yoyo"):
    print(t.text, t._.is_web)

best False
http://yoyo True


## Put it all together to simplify the text

In [18]:
from spacy.language import Language

In [19]:
try:
    Doc.remove_extension("simplify")
except:
    pass

In [20]:
def simplify(doc):
    # remove mentions, stopwords,, websites, set lowercase
    tokens = [t for t in doc if not (t._.is_mention or t.is_stop or t._.is_web)]
    # substitute emoji or lemma
    tokij = [t._.emoji_desc if t._.is_emoji else t.lemma_ for t in tokens]
    # join and split again (because emoji are translated to multiple words
    return " ".join(tokij)


Doc.set_extension("simplify", getter=simplify)

In [21]:
doc = nlpc("This is a test 😻 👍🏿")
doc, doc._.simplify

(This is a test 😻 👍🏿,
 'test smiling cat face with heart-eyes thumbs up dark skin tone')

# Sklearn models

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
preprocessor = PreProcessor(nlp=nlpc, classes=df_train.sentiment)

In [24]:
label_encoder = LabelEncoder()
label_encoder.fit(df_train.sentiment)

LabelEncoder()

In [25]:
def prepare_for_sklearn(df, label_encoder):
    docs = preprocessor.df_to_doc(df)
    X = [doc._.simplify for doc in docs]
    y = label_encoder.transform(df["sentiment"])

    return X, y

In [ ]:
X_train, y_train = prepare_for_sklearn(df_train, label_encoder)
X_test, y_test = prepare_for_sklearn(df_test, label_encoder)

In [ ]:
# Initialize a model collection

models = []

## MultinomialNB

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(ngram_range=(2, 2))

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

pipe_nb = Pipeline([("vectorizer", count_vector), ("classifier", nb)])

pipe_nb.fit(X_train, y_train)

In [ ]:
models.append((pipe_nb, "Naive Bayes"))

## LogReg


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

pipe_log = Pipeline([("vectorizer", count_vector), ("classifier", logreg)])

# model generation
pipe_log.fit(X_train, y_train)

In [ ]:
models.append((pipe_log, "Logistic Regression"))

In [ ]:
from IPython.display import Markdown, display

from sklearn.metrics import classification_report, f1_score


def show_score(y_true, y_pred, name):
    labels = label_encoder.classes_

    # f1 = f1_score(y_true, y_pred, labels=labels, average='weighted')

    # Model Accuracy
    display(Markdown(f"##{name} score"))
    print(classification_report(y_true, y_pred, target_names=labels))


for model, name in models:
    show_score(y_test, model.predict(X_test), name)

# Todo...

- What influence had the preprocessing on the results?
- TF/IDF
- Semantic extraction of tokens
- Try more effective models